In [1]:
import os
os.chdir('C:\Users\gaurav.singhal\Code\RSK')

In [2]:
from scipy import matrix
import scipy as sp
import matplotlib.pyplot as plt

In [5]:
import pdb

In [6]:
%load_ext autoreload 
%autoreload 2 

#from rsk.rsk import RSK
#from rsk.panel import PanelSeries

#%pdb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import csv
import scipy as sp
from functools import reduce

def is_numeric(entry):
    try:
        float(entry)
        return True
    except ValueError:
        return False

class PanelSeries:

    def __init__(self, panels, variable_names=None):
        '''
        A panel series is a time series of panels. Generally, it should not be constructed directly
        but rather from a builder method such as PanelSeries.from_csv
        :param panels:
        :param variable_names: names for the variables in the series.  The first two names are the names of time and group vars.
        '''

        # if the time index is numeric, need to convert to float so that data will be properly sorted
        if all([is_numeric(panel.time) for panel in panels]):
            for panel in panels:
                panel.time = float(panel.time)

        # sort panel by time variable
        self.data = sorted([(panel.time, panel) for panel in panels], key=lambda x: x[0])
        self.times = [time for time,panel in self.data]
        self.variable_names = variable_names

        # verify that we have balanced individuals
        groups = [group.name for group in self.data[0][1].data]
        if not all([groups == [group.name for group in panel.data] for (_,panel) in self.data]):
            raise ValueError("Currently, all panels must have the same members in the same order.")
        self.groups = groups

        #
        # compute group masks and check variables
        #
        group_counts_mask = []
        _, n_vars = self.data[0][1].data[0].data.shape
        self.n_variables = n_vars
        for (time,panel) in self.data:
            group_sizes = [group.size for group in panel.data]
            var_counts = [group.n_vars for group in panel.data]
            if not all([v==n_vars for v in var_counts]):
                raise ValueError("Must have same number of variables for each individual!")
            group_counts_mask.append(sp.diag(group_sizes))
        self.group_counts_mask = group_counts_mask

    def mean(self):
        '''
        Compute mean over entire panel series
        :return: vector of means
        '''
        pairs = [(panel.size(), panel.sum()) for _, panel in self.data]
        totals = reduce(lambda x,y: (x[0]+y[0], x[1] + y[1]), pairs)
        return totals[1]/totals[0]

    def means(self):
        '''
        compute group means matrix (n_groups x n_vars)
        :return: list of matrices by (n_groups x n_vars)
        '''
        return [panel.means() for _, panel in self.data]

    def cov(self):
        '''
        compute covariance matrix for each time slice across all groups
        :return: list of matrices (n_vars x n_vars)
        '''
        return [panel.cov() for t,panel in self.data]

    @staticmethod
    def from_csv(filename, time_index, group_index, log=True, header=True, drop_missing=True):
        '''
        Load a data set from a csv file.
        :param filename:
        :param time_index: column position of the time index
        :param group_index: column position of the group index
        :param header: treat first row as header
        :return:
        '''
        rowlist = []
        skip_index = {time_index, group_index}
        colnames = None
        with open(filename) as f:
            reader = csv.reader(f)
            if header:
                colnames = next(reader)
                colnames = [colnames[time_index], colnames[group_index]] + [entry for i,entry in enumerate(colnames) if i not in skip_index]

            for row in reader:
                if drop_missing and not all([is_numeric(entry) for i,entry in enumerate(row) if i not in skip_index]):
                    continue

                #put the time and group indices at start of row, followed by data
                if log:
                    rowlist.append([row[time_index], row[group_index]] + [sp.around(sp.log(float(entry)),5) for i,entry in enumerate(row) if i not in skip_index])                
                else:
                    rowlist.append([row[time_index], row[group_index]] + [float(entry) for i,entry in enumerate(row) if i not in skip_index])
        return PanelSeries.from_list(rowlist, colnames)

    @staticmethod
    def from_list(rowlist, variable_names = None):
        '''
        Construct a panel series from a list of rows (in an arbitrary order).  Each
        row must contain a time index and a group index at index 0 and 1 respectively.
        :param rowlist:
        :param colnames:
        :return: PanelSeries
        '''

        N = len(rowlist[0])
        if N<3:
            raise ValueError("Row must contain time, group, and at least one observation.")

        # collect panels out of rows (which are in arbitrary order)
        panel_dict = {}
        for row in rowlist:
            if len(row)!=N:
                raise ValueError("All rows in rowlist must have the same length.")
            t,g, data = row[0], row[1], row[2:]

            if t not in panel_dict:
                panel_dict[t] = {}
            panel = panel_dict[t]

            if g not in panel:
                panel[g] = []
            panel[g].append(data)

        # process each panel and order by time index
        panels = []
        for t,panel in sorted(panel_dict.items(), key=lambda x:x[0]):
            groups = []
            for g,data in sorted(panel.items(), key=lambda x:x[0]):
                groups.append(Group(g, data))
            panels.append(Panel(t, groups))

        return PanelSeries(panels, variable_names)


class Group:

    def __init__(self, name, observations):
        '''
        A group is a collection of observations on a set of individuals at a specific moment in time.
        :param name: label of the group
        :param observations: list of rows of observations of individuals or an n_individuals by n_vars matrix
        '''
        self.name = name
        if type(observations) is list:
            self.data = sp.array(observations)
        else:
            self.data = observations
        self.size = self.data.shape[0]
        self.n_vars = self.data.shape[1]

    def mean(self):
        return sp.mean(self.data, axis=0)

    def var(self):
        '''
        Group level variance
        :return:
        '''
        return sp.var(self.data, axis=0, ddof=1)

    def cov(self):
        '''
        Group level covariance
        :return: covariance matrix (n_vars x n_vars)
        '''
        return sp.cov(self.data, rowvar=False, ddof=1)

class Panel:

    def __init__(self, time, groups):
        '''
        A panel is a collection of groups obeserved at the same same moment in time.
        :param time: time at which the groups were observered
        :param groups: a list of groups
        '''
        self.time = time
        self.data = sorted(groups, key=lambda x: x.name)

    def size(self):
        return sum([group.size for group in self.data])

    def sum(self):
        return sum([group.mean()*group.size for group in self.data])

    def mean(self):
        return self.sum()/self.size()

    def means(self):
        return sp.vstack([group.mean() for group in self.data])

    def var(self):
        '''
        variance across all groups in this time slice
        :return:
        '''
        return sp.vstack([group.var() for group in self.data])

    def cov(self):
        '''
        covariance across all groups in this time slice
        :return: covariance matrix (n_vars x n_vars)
        '''
        M = sp.vstack([group.data for group in self.data])
        return sp.matrix(sp.cov(M, rowvar=False, ddof=1))


In [33]:
import warnings
import scipy as sp
import numpy as np
from scipy import transpose as t
from scipy.linalg import inv

class RSK:

    def __init__(self, transition_matrix, translation_matrix):
        '''
        :param transition_matrix:  array(n_alpha, n_alpha) transition model for latent alpha vector
        :param translation_matrix: array(n_vars, n_alpha) translation vector mapping alpha to means
        :return:
        '''
        self.translation_matrix = translation_matrix
        self.transition_matrix = transition_matrix

    def smooth(self, alpha, alpha_filter, V, V_filter):
        '''
        Backwards recursive smoother
        :param alpha:
        :param alpha_filter:
        :param V:
        :param V_filter:
        :return:
        '''
        n_periods, n_alpha, _ = V.shape
        alpha_smooth = sp.zeros(alpha.shape, )
        alpha_smooth[-1] = alpha_filter[-1]
        V_smooth = sp.zeros(V.shape, np.float64)
        V_smooth[-1] = V_filter[-1]
        B = sp.zeros(V.shape, np.float64)

        for i in range(n_periods-1,0,-1):
            B[i] = V_filter[i-1].dot(t(self.transition_matrix)).dot(inv(V[i]))
            alpha_smooth[i-1] = alpha_filter[i-1] + B[i].dot(alpha_smooth[i] - alpha[i])
            V_smooth[i-1] = V_filter[i-1] + B[i].dot(V_smooth[i]-V[i]).dot(t(B[i]))

        return alpha_smooth, V_smooth, B

    def fit(self, panel_series, a0, Q0, Q, smooth=True, sigma=None):
        '''
        Fit the RSK model to survey data
        :param panel_series: A PanelSeries object containing the survey data
        :param a0: array(n_alpha) initial value for the latent vector alpha
        :param Q0: array(n_alpha, n_alpha) Q0
        :param Q: array(n_alpha, n_alpha) Q
        :param smooth: boolean: apply the the smoothing algorithm
        :param sigma: specify a constant covariance matrix structure
        :return: alpha, alpha_filter, alpha_smooth, V, V_filter, V_smooth
        '''
        n_periods, n_vars = len(panel_series.times), panel_series.n_variables
        alpha, alpha_filter, alpha_smooth, _, _ , _,_ = self._fit(panel_series, a0, Q0, Q, smooth, sigma)

        # use smoothed values to make predictions?
        if smooth:
            alpha_pred = alpha_smooth
        else:
            alpha_pred = alpha_filter
        fitted_means = []
        for i in range(n_periods):
            n_groups = panel_series.group_counts_mask[i].shape[0]
            fitted_means.append(self.translation_matrix.dot(alpha_pred[i]).reshape(n_groups, n_vars))
        return fitted_means

    def _fit(self, panel_series, a0, Q0, Q, smooth=True, sigma=None):
        '''
        Fit the RSK model to survey data
        :param panel_series: A PanelSeries object containing the survey data
        :param a0: array(n_alpha) initial value for the latent vector alpha
        :param Q0: array(n_alpha, n_alpha) Q0
        :param Q: array(n_alpha, n_alpha) Q
        :param smooth: boolean: apply the the smoothing algorithm
        :param sigma: specify a constant covariance matrix structure
        :return: array(n_periods, n_vars) RSK estimated means
        '''

        # computations over the raw data
        n_periods, n_vars = len(panel_series.times), panel_series.n_variables
        y_means = panel_series.means()
        y_cov = panel_series.cov()

        # alpha hidden layer setup
        a0 = a0.reshape(-1,1)
        n_alpha = len(a0)
        alpha = sp.zeros((n_periods+1,  n_alpha, 1))
        alpha_filter = sp.zeros((n_periods+1, n_alpha, 1))
        alpha_filter[0] = a0

        # V covariance setup
        V = sp.zeros((n_periods + 1, n_alpha, n_alpha))
        V_filter = sp.zeros((n_periods+1, n_alpha, n_alpha))
        V_filter[0] = Q0

        # filter iterations
        transition_matrix, translation_matrix = self.transition_matrix, self.translation_matrix
        for i in range(1, n_periods+1):
            # compute group structure/covariance product

            if sigma is None:
                # no sigma provided, we use the covariance in the time slice
                _sigma = y_cov[i-1]
            elif len(sigma.shape)==3:
                # sigma is varying in time, pin it to the current time slice
                _sigma = sigma[i-1]
            else:
                # constant sigma specified
                _sigma = sigma
            try:
                ng_sigma_inv = sp.kron(panel_series.group_counts_mask[i-1], inv(_sigma))
            except Exception as e:
                print e.message, e.args
                pdb.set_trace()

            # predict
            alpha[i] = transition_matrix.dot(alpha_filter[i-1, :])
            V[i] = transition_matrix.dot(V_filter[i-1, :]).dot(t(transition_matrix)) + Q
            V_filter[i] = inv(inv(V[i]) + t(translation_matrix).dot(ng_sigma_inv).dot(translation_matrix))
            alpha_filter[i] = alpha[i] + V_filter[i].dot(t(translation_matrix)).dot(ng_sigma_inv).dot(y_means[i - 1].reshape(-1,1) - translation_matrix.dot(alpha[i]))

        if smooth:
            alpha_smooth, V_smooth, smoothing_matrix = self.smooth(alpha, alpha_filter, V, V_filter)
        else:
            alpha_smooth, V_smooth, smoothing_matrix = None, None, None

        return alpha, alpha_filter, alpha_smooth, V, V_filter, V_smooth, smoothing_matrix

    def fit_em(self, panel_series, a0, Q0, sigma0=None, constant_sigma=False, tolerance=1e-4, max_iters=500):
        '''
        Fit the RSK model to survey data
        :param panel_series: A PanelSeries object containing the survey data
        :param a0: array(n_alpha) initial value for the latent vector alpha
        :param Q0: array(n_alpha, n_alpha) Q0
        :param Q: array(n_alpha, n_alpha) Q
        :param sigma0: initial covariance structure. If none, the covariance of the panel_series is used
        :param tolerance: float
        :param max_iters: int
        :return: array(n_periods, n_vars) RSK estimated means
        '''

        n_periods, n_vars, n_alpha = len(panel_series.times), panel_series.n_variables, len(a0)
        Z,F = self.translation_matrix, self.transition_matrix
        Q = Q0
        sigma = sigma0
        error = 100 + tolerance
        alpha_stale = None
        iters = 0

        while error>tolerance and iters<max_iters:
            # fit alpha[0],...,alpha[T] the current values of the hyper parameters
            _, _, alpha_smooth, _, _, V_smooth, B = self._fit(panel_series, a0, Q0, Q, smooth=True, sigma=sigma)
            alpha = alpha_smooth
            V = V_smooth

            # update sigma
            sigma = sp.zeros((n_periods, n_vars, n_vars))
            for k, (time_label, panel) in enumerate(panel_series.data):
                n_groups = panel_series.group_counts_mask[k].shape[0]
                Nt = float(panel_series.group_counts_mask[k].sum())
                mu = Z.dot(alpha[k]).reshape((n_groups, n_vars))  # mu is a stacked vector
                ZVZ = Z.dot(V[k]).dot(t(Z))
                panel_mean = panel.mean()
                for j, group in enumerate(panel.data):
                    Ng = float(group.size)
                    diff1 = (group.mean() - mu[j]).reshape((-1,1))
                    diff2 = (panel_mean - mu[j]).reshape((-1,1))
                    idx = (j * n_vars, (j + 1) * n_vars)
                    sigma[k] += (Ng / Nt) * (group.cov() + diff1.dot(t(diff2)) + ZVZ[idx[0]:idx[1], idx[0]:idx[1]])
            
            if constant_sigma:
                sigma = sigma.mean(axis=0)
            
            if sp.all(sigma==0):
                pdb.set_trace()
                
            # update Q
            Q = sp.zeros((n_alpha, n_alpha))
            N = sum([panel.size() for _,panel in panel_series.data])
            for k in range(n_periods):
                Nt = panel_series.group_counts_mask[k].sum()
                diff = alpha[k+1] - F.dot(alpha[k])
                Q += (Nt/N) * (diff.dot(t(diff)) + V[k+1] + F.dot(V[k]).dot(t(F)) - F.dot(B[k+1]).dot(V[k+1]) - V[k+1].dot(B[k+1]).dot(t(F)))

            # update a0, Q0
            a0 = alpha[0]
            Q0 = V[0]

            # compute the error and prepare for next step
            if alpha_stale is None:
                error = tolerance + 1000
            else:
                error = np.linalg.norm(alpha.reshape((-1,)) - alpha_stale.reshape((-1,)))
                print("Iteration %d, error: %.8f" % (iters,error))
            alpha_stale = alpha
            iters += 1

        if iters==max_iters:
            warnings.warn("RSK EM algorithm failed to converge before max iterations %d reached.  Current error=%.8f" % (max_iters, error))
        else:
            print("Converged in %d iterations..." % iters)
        return self.fit(panel_series, a0, Q0, Q, smooth=True, sigma=sigma), sigma


In [17]:
panel_series = PanelSeries.from_csv(r'C:\Users\gaurav.singhal\Code\RSK\test\ebola\LiberiaPrices.csv', 0, 1)

In [18]:
print("Analysis of mVAMPrices Ebola data: ")
print("VARS: %s" % str(panel_series.variable_names))
print("GROUPS: %s" % str(panel_series.groups))
print("TIMES: %s" % str(panel_series.times))

Analysis of mVAMPrices Ebola data: 
VARS: ['Month', 'StatisticalAggregation', 'PalmOil_1L', 'LocalRice_1cup', 'ImportRice_1cup', 'ManLbr_Daily']
GROUPS: ['Bong', 'Grand Bassa', 'Lofa', 'Margibi', 'Montserrado', 'Nimba', 'Southeast w/o Bassa', 'Western']
TIMES: [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0, 17.0]


In [19]:
n_vars = len(panel_series.variable_names)-2
n_groups = len(panel_series.groups)

In [20]:
# Specify a slow moving transition matrix for alpha
n_alpha = 64        # for this implementation, we need n_alpha>n_groups*n_vars
delta = 0.01       # off diagonal transition probability
transition_matrix = (1-(n_alpha-1)*delta)*sp.eye(n_alpha) + (delta*sp.ones((n_alpha,n_alpha)) - delta*sp.eye(n_alpha))

In [21]:
# random translation matrix
translation_matrix = matrix(sp.random.uniform(size=(n_groups*n_vars, n_alpha)))

In [22]:
# solve for a0 so that the estimated mean at time 0 is mu[0]
means = panel_series.means()
# solve via generalized inverse since this system is underdetermined
a0 = sp.linalg.pinv(translation_matrix).dot(means[0].reshape(-1,1)) 
a0 = sp.linalg.solve(transition_matrix, matrix(a0))

In [23]:
# specify an alpha covariance structure
Q =  matrix(sp.eye(n_alpha))
Q0 = matrix(sp.eye(n_alpha))

In [59]:
# fit RSK
rsk_filter = RSK(transition_matrix, translation_matrix)
fitted_means, sigma = rsk_filter.fit_em(panel_series, a0, Q0)

Iteration 1, error: 30.70060007
Iteration 2, error: 3.24188689
Iteration 3, error: 2.22469031
Iteration 4, error: 1.75634065
Iteration 5, error: 1.48538150
Iteration 6, error: 1.30868066
Iteration 7, error: 1.18475073
Iteration 8, error: 1.09354691
Iteration 9, error: 1.02415341
Iteration 10, error: 0.97010053
Iteration 11, error: 0.92730360
Iteration 12, error: 0.89305443
Iteration 13, error: 0.86548627
Iteration 14, error: 0.84327129
Iteration 15, error: 0.82544028
Iteration 16, error: 0.81127060
Iteration 17, error: 0.80021390
Iteration 18, error: 0.79184798
Iteration 19, error: 0.78584393
Iteration 20, error: 0.78194300
Iteration 21, error: 0.77994012
Iteration 22, error: 0.77967174
Iteration 23, error: 0.78100673
Iteration 24, error: 0.78383940
Iteration 25, error: 0.78808398
Iteration 26, error: 0.79367009
Iteration 27, error: 0.80053887
Iteration 28, error: 0.80863948
Iteration 29, error: 0.81792585
Iteration 30, error: 0.82835333
Iteration 31, error: 0.83987518
Iteration 32, er

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:198: UserWarning: RSK EM algorithm failed to converge before max iterations 500 reached.  Current error=0.08809175


In [60]:
for i,naive_mean, rsk_mean in zip(panel_series.times, means,fitted_means):
    print("At time %d: "  % i)
    print("Naive sample mean: \n%s" % naive_mean)
    print("RSK mean: \n%s" % rsk_mean)

At time 2: 
Naive sample mean: 
[[ 3.1113727   3.2039077   3.2411167   5.0381479 ]
 [ 3.18798     3.2627053   3.2515371   5.2655124 ]
 [ 3.2483879   3.2416263   3.2824234   5.0632968 ]
 [ 3.1395389   3.2526416   3.2590484   5.3126991 ]
 [ 3.25811494  3.29133017  3.2717724   5.33485863]
 [ 3.1672262   3.1879772   3.2943414   5.143365  ]
 [ 3.34482233  3.35177573  3.41112887  5.254625  ]
 [ 3.2397298   3.2879222   3.2962205   5.3914025 ]]
RSK mean: 
[[  1.65927571  -0.8257848   -5.24538062  77.63578186]
 [ -4.83330332  -1.02541241  -9.27283863  93.86986483]
 [-13.24335782 -17.74231489   3.72474304  62.98194016]
 [ -7.09480663  -9.19322696  -3.25186807  79.98532995]
 [  7.35946553   6.77015018  -6.82124536  85.35149537]
 [ -7.84421493 -10.00956636   1.09650718  67.1247752 ]
 [ -5.00429029  -6.2213859   -1.19926197  82.32339624]
 [-13.09901453 -11.51469929  -1.17243643  79.47042991]]
At time 3: 
Naive sample mean: 
[[ 3.1150915   3.2001139   3.2349219   5.0880924 ]
 [ 3.17418235  3.2709337

In [62]:
import pandas as pd

L = n_groups*len(panel_series.times)
times = sp.reshape(sp.sort(sp.tile(panel_series.times,n_groups)),L,1)
groups = sp.reshape(sp.tile(panel_series.groups,len(panel_series.times)),L,1)
naive_means = sp.column_stack((times,groups,sp.vstack(means)))
rsk_means = sp.column_stack((times,groups,sp.vstack(fitted_means)))

In [66]:
%matplotlib auto

plt.clf()
plt.close()
f, axArray = plt.subplots(4, sharex=True)

colors = dict(zip(panel_series.groups,['b','g','r','c','m','k','darkgreen','darkgray']))

grouped = pd.DataFrame(naive_means,columns=panel_series.variable_names).convert_objects(convert_numeric=True).groupby('StatisticalAggregation')
for key, group in grouped:
    group.plot(ax=axArray[0], kind='line', x='Month', y='ManLbr_Daily', label=key, legend=False, color=colors[key],ls='dotted')
    group.plot(ax=axArray[1], kind='line', x='Month', y='PalmOil_1L', label=key, legend=False, color=colors[key],ls='dotted')
    group.plot(ax=axArray[2], kind='line', x='Month', y='LocalRice_1cup', label=key, legend=False, color=colors[key],ls='dotted')
    group.plot(ax=axArray[3], kind='line', x='Month', y='ImportRice_1cup', label=key, legend=False, color=colors[key],ls='dotted')
    
grouped = pd.DataFrame(rsk_means,columns=panel_series.variable_names).convert_objects(convert_numeric=True).groupby('StatisticalAggregation')
for key, group in grouped:
    group.plot(ax=axArray[0], kind='line', x='Month', y='ManLbr_Daily', label=key, legend=False, color=colors[key],ls='solid')
    group.plot(ax=axArray[1], kind='line', x='Month', y='PalmOil_1L', label=key, legend=False, color=colors[key],ls='solid')
    group.plot(ax=axArray[2], kind='line', x='Month', y='LocalRice_1cup', label=key, legend=False, color=colors[key],ls='solid')
    group.plot(ax=axArray[3], kind='line', x='Month', y='ImportRice_1cup', label=key, legend=False, color=colors[key],ls='solid')

axArray[0].set_title('ManLbr')
axArray[1].set_title('PalmOil')
axArray[2].set_title('LocalRice')
axArray[3].set_title('ImportRice')
    
markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colors.values()]
plt.legend(markers, colors.keys(), numpoints=1,loc=2,bbox_to_anchor=(1.05, 4), borderaxespad=0)
plt.show()

Using matplotlib backend: Qt5Agg


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [37]:
fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)

df = pd.DataFrame(naive_means,columns=panel_series.variable_names).convert_objects(convert_numeric=True)
colors = dict(zip(panel_series.groups,['b','g','r','c','m','k','darkgreen','darkgray']))
df['Color'] = df['StatisticalAggregation'].map(colors)
ax1.plot(df['Month'], df['ManLbr_Daily'],  c = df['Color'], alpha = 0.8)


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


ValueError: to_rgba: Invalid rgba arg "0              b
1              g
2              r
3              c
4              m
5              k
6      darkgreen
7       darkgray
8              b
9              g
10             r
11             c
12             m
13             k
14     darkgreen
15      darkgray
16             b
17             g
18             r
19             c
20             m
21             k
22     darkgreen
23      darkgray
24             b
25             g
26             r
27             c
28             m
29             k
         ...    
74             r
75             c
76             m
77             k
78     darkgreen
79      darkgray
80             b
81             g
82             r
83             c
84             m
85             k
86     darkgreen
87      darkgray
88             b
89             g
90             r
91             c
92             m
93             k
94     darkgreen
95      darkgray
96             b
97             g
98             r
99       

> c:\program files\anaconda2\lib\site-packages\matplotlib\colors.py(376)to_rgba()
    374         except (TypeError, ValueError) as exc:
    375             raise ValueError(
--> 376                 'to_rgba: Invalid rgba arg "%s"\n%s' % (str(arg), exc))
    377 
    378     def to_rgba_array(self, c, alpha=None):

ipdb> exit()
